# Baromètre des résultats - PDF reports

## Imports

In [1]:
import json
import os
from os.path import join
import datetime
import random

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import numpy as np

from fpdf import FPDF

## Functions

In [2]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [3]:
def get_dep_infos(dep, dep_taxo_path='./data/taxonomies/departements.json', reg_taxo_path='./data/taxonomies/regions.json') :
    with open(dep_taxo_path, 'r') as f :
        dep_dict_list = json.loads(f.read())
    with open(reg_taxo_path, 'r') as f :
        reg_dict_list = json.loads(f.read())
        
    dep_dict = [dict_ for dict_ in dep_dict_list if dict_['dep'] == dep][0]
    dep_name = dep_dict['libelle']
    reg = dep_dict['reg']
    
    reg_dict = [dict_ for dict_ in reg_dict_list if dict_['reg'] == reg][0]
    reg_name = reg_dict['libelle']
    
    res = dict(dep=dep,
               dep_name=dep_name,
               reg=reg,
               reg_name=reg_name
              )
    
    return res

In [4]:
def get_dep_synthese_data(dep, ovq_synthese_data) :
    for dep_dict in ovq_synthese_data :
        if dep_dict['dep'] == dep :
            res = dep_dict
            break
    return res

In [5]:
def get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data) :
    dep_dict = get_dep_synthese_data(dep, ovq_synthese_data)
    
    for ovq_dict in dep_dict['ovq'] :
        for ind_dict in ovq_dict['indicateurs'] :
            if id_indicateur in ind_dict.keys() :
                res = ind_dict[id_indicateur][0]
                break
    return res

In [6]:
def get_indicateur_structure_cible(id_indicateur, structure_cible_data) :
    for ovq_dict in structure_cible_data :
        for ind_dict in ovq_dict['indicateurs'] :
            if ind_dict['id_indicateur_fr'] == id_indicateur :
                res = ind_dict
                break
    return res

In [7]:
def get_ovq_structure_cible(id_ovq, structure_cible_data) :
    for ovq_dict in structure_cible_data :
        if ovq_dict['id_ovq'] == id_ovq :
            res = ovq_dict
            break
    return res

In [8]:
def make_pct_bullet_chart(pct, bar_color='#3D49A5', bg_color='#BBBFDF') :
    
    if pct < 0 : #To improve if inverted bullet chart is possible with plotly
        bar_color = '#ED782F'
        bg_color = '#ED782F'
    
    rounded_pct = round(pct)
    
    fig = go.Figure(go.Indicator(
        mode = "gauge",
        gauge = {'shape': "bullet", 'axis': {'visible':False, 'range': [0, 100]}, 'bgcolor':bg_color, 'bordercolor':'white', 'bar':{'thickness':1, 'color':bar_color}},
        value = rounded_pct,
        number = {'suffix':'%'},
        domain = {'x': [0, 1], 'y': [0, 1]}))
    
    fig.update_layout(height = 400, width=2400, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

    fig.add_annotation(dict(font=dict(color='white',size=100),
                                            x=0.05,
                                            y=0.5,
                                            showarrow=False,
                                            text=str(rounded_pct)+'%',
                                            textangle=0,
                                            xanchor='left',
                                            xref="paper",
                                            yref="paper"))

    return fig

In [9]:
months_fr_dict = {
    '01':'janvier',
    '02':'février',
    '03':'mars',
    '04':'avril',
    '05':'mai',
    '06':'juin',
    '07':'juillet',
    '08':'août',
    '09':'septembre',
    '10':'octobre',
    '11':'novembre',
    '12':'décembre'
}

# yearmonth should be in '2020-01' format

def yearmonth_to_fr(yearmonth) :
    month = yearmonth[-2:]
    year = yearmonth[:4]
    
    return months_fr_dict[month] + ' ' + year

In [10]:
def make_detail_chart(df, chart_type) :
    if chart_type == 'histo' :
        fig = px.bar(df, x='date', y='value',
                         labels={'date':'', 'value':''}, 
                         height=400,
                         width=900,
                         color=None,
                         orientation='v',
                         text=None,
                         title=None)
        fig.update_layout(showlegend=False)
        fig.layout['xaxis'].showgrid = False
        fig.update_traces(marker_color='#00AC8C')
        fig.update_xaxes(tickangle=-45,
                         tickfont=dict(size=8),
                         tickmode = 'array',
                         tickvals = list(df['date']),
                         ticktext= list(df['date_tick']))
                        
        return fig
    
    elif chart_type == 'line' :
        fig = px.line(df, x='date', y='value',
                         labels={'date':'', 'value':''}, 
                         height=400,
                         width=750,
                         color=None,
                         orientation='v',
                         text=None,
                         title=None)
        fig.update_layout(showlegend=False)
        fig.layout['xaxis'].showgrid = False
        fig.update_traces(marker=dict(size=12, line=dict(width=2, color='white')), marker_color='#00AC8C', line_color='#00AC8C', line_width=6)
        fig.update_xaxes(tickangle=-45,
                         tickfont=dict(size=8),
                         tickmode = 'array',
                         tickvals = list(df['date']),
                         ticktext= list(df['date_tick']))
        fig.data[0].update(mode='markers+lines')

        return fig
    
    else :
        print('Unknown chart type.')
        return None

## Parameters

In [11]:
ovq_data_folder_path = './ovq-data'

ovq_synthese_data_path = os.path.join(ovq_data_folder_path, 'prod', 'par_territoire', 'territoire-departemental-global-light.json')
ovq_structure_cible_path = os.path.join(ovq_data_folder_path, 'scripts', 'structure-cible.json')
ovq_structure_families_path = os.path.join(ovq_data_folder_path, 'scripts', 'structure-families.json')

img_dir_path = './img/'
reports_dir_path = './reports/'

In [12]:
cat_mesures_dict = {
    'Transition écologique' : ["Déployer le plan vélo",
                               "Mettre en œuvre la sortie du plastique à usage unique et lutter contre le gaspillage",
                               "Verdir le parc automobile",
                               "Instaurer des zones à faibles émissions"
                              ],
    'Économie - Emploi' : ["Développer l’apprentissage",
                           "Plan #1jeune1solution",
                           "Supprimer la taxe d’habitation sur les résidences principales",
                          ],
    'Santé - Famille - Handicap' : ["Doubler le nombre de maisons de santé",
                                    "Allonger le congé paternité pour un meilleur développement de l’enfant",
                                    "Simplifier l’accès aux droits des personnes handicapées",
                                    "Proposer une offre de lunettes, appareils auditifs et prothèses dentaires remboursée à 100%"
                                   ],
    'Logement' : ['Offrir un logement aux sans-abris : Logement d’abord'],
    'Éducation' : ["Offrir une scolarisation inclusive à tous les enfants handicapés",
                   "Déployer Parcoursup ",
                   "Limiter les classes à 24 en grande section, CP, CE1",
                   "Dédoubler les classes en REP (grande section, CP, CE1)"
                  ],
    'Sécurité' : ["Lutter contre les stupéfiants",
                  "Lutter contre les atteintes aux principes républicains",
                  "Lutter contre les violences faites aux femmes",
                  "Réduire la mortalité sur les routes",
                  "Renforcer la sécurité du quotidien"
                 ],
    'Culture' : ["Déployer le Pass culture"],
    'Services publics et territoires' : ["Assurer une bonne couverture en internet fixe et en téléphonie mobile pour tous les Français d'ici 2022",
                                         "Déployer une offre France Services dans tous les territoires ",
                                         "Améliorer la qualité du service rendu à l’usager"
                                        ]
}

## Building parameters from data

In [13]:
#Loading OVQ synthese data
with open(ovq_synthese_data_path, 'r') as f :
    ovq_synthese_data = json.loads(f.read())

In [14]:
#Departements list
dep_list = sorted(list(set([dep_dict['dep'] for dep_dict in ovq_synthese_data])))
print(len(dep_list))

101


In [15]:
#Indicateurs list
id_indicateur_list = []
for dep_dict in ovq_synthese_data :
    for ovq_dict in dep_dict['ovq'] :
        for ind_dict in ovq_dict['indicateurs'] :
            id_indicateur_list += list(ind_dict.keys())
            
id_indicateur_list = list(set(id_indicateur_list))
print(len(id_indicateur_list))

30


In [16]:
#Loading structure data

with open(ovq_structure_cible_path, 'r') as f :
    structure_cible_data = json.loads(f.read())
    
with open(ovq_structure_families_path, 'r') as f :
    structure_families_data = json.loads(f.read())

## Making the graphs

In [17]:
mkdir_ifnotexist(join(img_dir_path, 'graphs'))

### Synthese gauges

In [18]:
%%time
for dep in dep_list :
    for id_indicateur in id_indicateur_list :
        data = get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data)
        pct = data['target_percentage']
        
        if pct is not None :
            fig = make_pct_bullet_chart(pct)

            if fig is not None :
                img_path = join(img_dir_path, 'graphs', 'bullet_chart-synthese-target_percentage-{}-dep_{}.png'.format(id_indicateur, dep))
                fig.write_image(img_path)

CPU times: user 9.2 s, sys: 770 ms, total: 9.97 s
Wall time: 34.9 s


### Detail graphs

In [39]:
%%time
detail_data_folder = os.path.join(ovq_data_folder_path, 'prod', 'par_thematique')

for thematique in os.listdir(detail_data_folder) :
    if os.path.isdir(os.path.join(detail_data_folder, thematique)) :
        for ovq in os.listdir(os.path.join(detail_data_folder, thematique)) :
            json_files_list = os.listdir(os.path.join(detail_data_folder, thematique, ovq))

            for dep in dep_list :
                
                #Following indicateur done or not
                id_indicateur_done_list = []
                
                for level in ['departemental', 'regional', 'national'] :
                    
                    json_filename = [filename for filename in json_files_list if filename.endswith('-{}.json'.format(level))][0]
                    json_file_path = os.path.join(detail_data_folder, thematique, ovq, json_filename)

                    with open(json_file_path, 'r') as f :
                        data = json.loads(f.read())

                    if level == 'departemental' :
                        for dict_ in data :
                            if dict_['dep'] == dep :
                                lvl_dict = dict_
                                break
                    
                    elif level == 'regional' :
                        reg = get_dep_infos(dep)['reg']
                        for dict_ in data :
                            if dict_['reg'] == reg :
                                lvl_dict = dict_
                                break
                                
                    elif level == 'national' :
                        lvl_dict = data[0]

                    ovq_indicateur_list = list(lvl_dict['values'].keys())
                    
                    for id_indicateur in ovq_indicateur_list :
                        if (id_indicateur in id_indicateur_list) and (id_indicateur not in id_indicateur_done_list) :
                            chart_type = get_indicateur_structure_cible(id_indicateur, structure_cible_data)['odm_chart_type']

                            df_ind = pd.DataFrame(lvl_dict[id_indicateur])
                            df_ind = df_ind.sort_values('date')
                            df_ind['date_tick'] = df_ind['date'].apply(lambda x : yearmonth_to_fr(x[:7]))

                            fig = make_detail_chart(df_ind, chart_type)
                            img_path = join(img_dir_path, 'graphs', 'graph-detail-{}-dep_{}.png'.format(id_indicateur, dep))

                            if fig is not None :
                                fig.write_image(img_path)
                                
                            id_indicateur_done_list += [id_indicateur]

        print('{} - {} done.'.format(datetime.datetime.today(), thematique))

2021-01-13 18:31:30.900546 - handicap done.
2021-01-13 18:32:15.780092 - sante-famille done.
2021-01-13 18:32:59.347703 - securite done.
2021-01-13 18:33:19.242559 - relance-emploi-economie done.
2021-01-13 18:33:39.864309 - education done.
2021-01-13 18:34:00.936245 - services-publics-et-territoires done.
2021-01-13 18:34:16.492159 - logement done.
2021-01-13 18:34:44.947674 - transition-ecologique done.
2021-01-13 18:34:51.754319 - jeunesse-culture-engagement done.
2021-01-13 18:34:59.498648 - culture done.
CPU times: user 2min 21s, sys: 2.46 s, total: 2min 24s
Wall time: 3min 42s


In [36]:
id_indicateur

'nb-beneficiaires-siae'

In [38]:
len(id_indicateur_list)

30

## Building the reports

In [40]:
mkdir_ifnotexist(reports_dir_path)

In [41]:
global title_header
title_header = ''
#global subtitle_header
#subtitle_header = ''

In [42]:
class PDF(FPDF):
    def header(self):
        if (self.page_no() != 1 ):
            # Logo
            self.image(os.path.join(img_dir_path, 'gouv.png'), 10, 8, 45)
            # Arial bold 15
            self.cell(50)
            self.set_font('Arial', 'B', 16)
            self.cell(80, 15, title_header, 0, 1, 'A')

            # Move to the right
            # Title
            self.set_font('Arial', 'I', 9)
            self.cell(50)
            #self.cell(50, 10, subtitle_header+' - aides-entreprises.data.gouv.fr', 0, 1, 'A', link='https://aides-entreprises.data.gouv.fr/')
            self.cell(50, 10, 'barometre-resultats.data.gouv.fr', 0, 1, 'A', link='https://barometre-resultats.data.gouv.fr/')
            # Line break  
            pdf.line(40, 38, 170, 38)
            self.ln(7)
        
    # Page footer
    def footer(self):
        if (self.page_no() != 1 ):
            # Position at 1.5 cm from bottom
            self.set_y(-15)
            # Arial italic 8
            self.set_font('Arial', 'I', 8)
            # Page number
            
            self.cell(0, 10, 'Direction Interministérielle du Numérique (DINUM) - Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')
            
    def chapter_title(self, num, label):
        # Arial 12
        self.set_font('Arial', '', 12)
        # Background color
        self.set_fill_color(200, 220, 255)
        # Title
        self.cell(0, 6, 'Chapter %d : %s' % (num, label), 0, 1, 'L', 1)
        # Line break
        self.ln(4)
    
    def chapter_body(self, name):
        # Read text file
        with open(name, 'rb') as fh:
            txt = fh.read().decode('latin-1')
        # Times 12
        self.set_font('Times', '', 12)
        # Output justified text
        self.multi_cell(0, 5, txt)
        # Line break
        self.ln()

    def print_chapter(self, num, title, name):
        self.add_page()
        self.chapter_title(num, title)
        self.chapter_body(name)

In [43]:
def format_val(val,add):
    return '{:,}'.format(int(float(val))).replace(',', ' ')+add

In [44]:
def insert_indicateur(pdf, dep, ind_data, ind_struc) :
    
    #Getting relevant information in data
    initial_value = ind_data['initial_value']
    initial_value_date = ind_data['initial_value_date']
    latest_value = ind_data['latest_value']
    latest_value_date = ind_data['latest_value_date']
    target_value = ind_data['target']
    target_percentage = ind_data['target_percentage']
    target_value_date = ind_data['target_date']
    progression_percentage = ind_data['progression_percentage']
    from_level = ind_data['from_level']
    
    #Getting relevant information in structure
    id_indicateur = ind_struc['id_indicateur_fr']
    nom_indicateur = ind_struc['nom_indicateur'].replace('’', "'")
    odm_text = ind_struc['odm_text']
    kpi_format = ind_struc['odm_kpi_format']
    kpi_unit = ind_struc['odm_kpi_unit']
    chart_unit = ind_struc['odm_chart_unit']
    to_round = ind_struc['toRound']
    
    if to_round is not None :
        if initial_value is not None :
            initial_value = round(initial_value, to_round)
        if latest_value is not None :
            latest_value = round(latest_value, to_round)
        if target_value is not None :
            target_value = round(target_value, to_round)
    
    #Initializing blocks parameters
    nb_blocks = 1
    ini_block = False
    tar_block = False
    
    #Current value (always present)
    latest_value_date_str = yearmonth_to_fr(latest_value_date[:7])
    latest_value_str = str(latest_value).replace('.',',')
    
    #Insert initial block if init value exists
    if initial_value is not None :
        initial_value_date_str = yearmonth_to_fr(initial_value_date[:7])
        initial_value_str = str(initial_value).replace('.',',')
        nb_blocks += 1
        ini_block = True
        
    #Insert target block if target value exists
    if target_value is not None :
        target_value_date_str = yearmonth_to_fr(target_value_date[:7])
        target_value_str = str(target_value).replace('.',',')
        nb_blocks += 1
        tar_block = True
        
    #Blocks settings depending on existing data
    block_spacing = 5
    block_width = (190 - (nb_blocks-1)*block_spacing)/nb_blocks
    
    #INDICATOR DISPLAY
    
    #Indicator title
    pdf.set_text_color(0,0,0)
    pdf.set_font('Arial', 'B', 12)
    pdf.multi_cell(190, 8, 'Indicateur : '+ nom_indicateur, align='L')
    pdf.ln(5)
    
    #Information section
    y_info_top = pdf.get_y()
    info_img_size = 7
    
    pdf.set_fill_color(83,101,125)
    pdf.rect(20,y_info_top,180,14,'F')
    pdf.image(os.path.join(img_dir_path, 'information.png'), x=10, y=y_info_top+info_img_size/2, w=info_img_size)
    pdf.set_font('Arial', 'I', 7)
    pdf.set_text_color(255,255,255)
    pdf.cell(10)
    pdf.multi_cell(100, 5, txt="INFO")
    pdf.ln(12)
    
    #Data geo level indication
    if from_level == 'dep' :
        from_level_str = "Les résultats pour le département : " + get_dep_infos(dep)['dep_name']
    elif from_level == 'reg' :
        from_level_str = "Les résultats pour la région : " + get_dep_infos(dep)['reg_name']
    elif from_level == 'nat' :
        from_level_str = "Les résultats pour la France"
    
    pdf.set_text_color(0,0,0)
    pdf.set_font('Arial', '', 10)
    pdf.set_fill_color(245,245,245);
    pdf.cell(190, 8, from_level_str, 0, 2, 'L', fill=True)
    pdf.ln(3)
    
    #Indicator blocks
    
    y_blocks_top = pdf.get_y()
    x_blocks_left = pdf.get_x()
    
    #Initial value block
    if ini_block == True :
        pdf.set_fill_color(253, 244, 242)
        pdf.set_font('Arial', '', 8)
        pdf.cell(block_width, 8, 'En {}'.format(initial_value_date_str), 0, 1, 'L', fill=True)
        pdf.set_fill_color(254, 249, 248)
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(block_width, 10, initial_value_str, 0, 1, 'L', fill=True)
        pdf.set_font('Arial', '', 10)
        pdf.cell(block_width, 3, kpi_unit, 0, 1, 'L', fill=True)
        pdf.cell(block_width, 10, '', 0, 1, 'L', fill=True)
    
    #Current value block (always here)
    if nb_blocks > 1 :
        pdf.set_xy(block_width + pdf.l_margin + block_spacing, y_blocks_top)
    pdf.set_fill_color(234, 244, 239)
    pdf.set_font('Arial', '', 8)
    pdf.cell(block_width, 8, 'En {}'.format(latest_value_date_str), 0, 2, 'L', fill=True)
    pdf.set_fill_color(244, 250, 247)
    pdf.set_font('Arial', 'B', 20)
    pdf.cell(block_width, 10, latest_value_str, 0, 2, 'L', fill=True)
    pdf.set_font('Arial', '', 10)
    pdf.cell(block_width, 3, kpi_unit, 0, 2, 'L', fill=True)
    pdf.cell(block_width, 10, '', 0, 2, 'L', fill=True)
    
    #Target value block
    if tar_block == True :
        if nb_blocks == 2 :
            pdf.set_xy(block_width + pdf.l_margin + block_spacing, y_blocks_top)
        elif nb_blocks == 3 :
            pdf.set_xy(2*block_width + pdf.l_margin + 2*block_spacing, y_blocks_top)
            
        pdf.set_fill_color(229, 229, 243)
        pdf.set_font('Arial', '', 8)
        pdf.cell(block_width, 8, 'Cible {}'.format(target_value_date_str), 0, 2, 'L', fill=True)
        pdf.set_fill_color(242, 242, 248)
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(block_width, 10, target_value_str, 0, 2, 'L', fill=True)
        pdf.set_font('Arial', '', 10)
        pdf.cell(block_width, 3, kpi_unit, 0, 2, 'L', fill=True)
        
        #Inserting bullet chart (prog %)
        block_left_x = pdf.get_x()
        block_bottom_y = pdf.get_y()
        pdf.cell(block_width, 10, '', 0, 2, 'L', fill=True)
        bullet_chart_width = 60
        pdf.set_xy(block_left_x + block_width/2 - bullet_chart_width/2, block_bottom_y)
        prog_img_path = join(img_dir_path, 'graphs', 'bullet_chart-synthese-target_percentage-{}-dep_{}.png'.format(id_indicateur, dep))
        pdf.image(prog_img_path, w=bullet_chart_width)
    
    #CHART DISPLAY
    try :
        pdf.set_x(x_blocks_left)
        chart_width = 120
        chart_img_path = join(img_dir_path, 'graphs', 'graph-detail-{}-dep_{}.png'.format(id_indicateur, dep))
        pdf.image(chart_img_path, w=chart_width, x=x_blocks_left + 190/2 - chart_width/2)

        pdf.set_font('Arial', 'I', 8)
        pdf.cell(190, 8, 'Unité : {}'.format(kpi_unit), 0, 2, 'C')
    except :
        print('No chart found for {}'.format(id_indicateur))

    return pdf

In [50]:
%%time

mkdir_ifnotexist(os.path.join(reports_dir_path, 'pdf'))

for dep in ['01']:
    
    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()

    #Logos
    pdf.image(os.path.join(img_dir_path, 'gouv.png'), 10, 8, 125)
    
    # Arial bold 15
    pdf.set_font('Arial', 'B', 28)
    # Move to the right
    pdf.cell(50)
    pdf.ln(100)
    
    # Title
    pdf.cell(10)
    pdf.cell(50, 10, 'BAROMÈTRE DES RÉSULTATS DE', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, "L'ACTION PUBLIQUE", 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(60)

    pdf.set_font('Arial', 'I', 20)
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, 'Présentation des résultats pour le département :', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, dep + ' - ' + get_dep_infos(dep)['dep_name'], 0, 1, 'A')
    pdf.ln(70)

    pdf.set_font('Arial', 'I', 8)

    pdf.cell(10)
    pdf.cell(50, 10, "Données issues du baromètre des résultats consultable sur https://barometre-resultats.data.gouv.fr/", 0, 1, 'A')
    
    #Indicateurs
    
    #subtitle_header = find_last_update('aides')
    
    for thematique_dict in structure_families_data :
    
        thematique = thematique_dict['nom_ovq']
        title_header = thematique.upper()

        id_ovq_list = thematique_dict['id_ovq']
        for id_ovq in id_ovq_list :
            try :
                pdf.add_page()
                ovq_dict = get_ovq_structure_cible(id_ovq, structure_cible_data)
                nom_ovq = ovq_dict['nom_ovq'].replace('’', "'").replace('œ','oe')
                pdf.set_font('Arial', 'B', 14)
                pdf.multi_cell(190, 8, nom_ovq, align='L')
                #Get Txt info OVQ

                ovq_id_indicateur_list = [ind_dict['id_indicateur_fr'] for ind_dict in ovq_dict['indicateurs']]

                add_page_for_ind = False
                for id_indicateur in ovq_id_indicateur_list :
                    if add_page_for_ind == True :
                        pdf.add_page()
                    ind_data = get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data)
                    ind_struc = get_indicateur_structure_cible(id_indicateur, structure_cible_data)

                    pdf = insert_indicateur(pdf, dep, ind_data, ind_struc)
                    add_page_for_ind = True
            except :
                print('No structure cible for {}'.format(id_ovq))
    
    pdf.output(os.path.join(reports_dir_path, 'pdf', 'Baromètre_résultats_'+dep+'.pdf'), 'F')
    
    print(str(datetime.datetime.today()) + ' - ' + dep + ' done.')

No structure cible for OVQ-EAC
No structure cible for OVQ-REL
2021-01-13 18:46:42.164093 - 01 done.
CPU times: user 23.4 s, sys: 8.07 s, total: 31.4 s
Wall time: 31.5 s
